In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import raster_tools as rt
import os
import sys
import geopandas as gpd

# read the mtbs perims at ../FireLab/Project/data/MTBS_Data/mtbs_perims/mtbs_perims_DD_v2.shp
# mtbs_perims = gpd.read_file('data/mtbs_perims/mtbs_perimeter_data/mtbs_perims_DD.shp')
# nifc_perims_2022 = gpd.read_file('data/nifc_perims/nifc_perims_2022/nifc_perims_2022.shp')
usfs_perims_2022 = gpd.read_file('data/usfs_perims/usfs_perims/2022/usfs_perims_2022.shp')
usfs_perims_2023 = gpd.read_file('data/usfs_perims/usfs_perims/2023/usfs_perims_2023.shp')

In [12]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

#### USFS Perims

In [37]:
usfs_perims_2022 = usfs_perims_2022[usfs_perims_2022['TOTALACRES'] > 2480]
usfs_perims_2023 = usfs_perims_2023[usfs_perims_2023['TOTALACRES'] > 2480]

In [38]:
usfs_perims_2022['Ig_Date'] = pd.to_datetime(usfs_perims_2022['DISCOVERYD'])
usfs_perims_2022['Ig_Date'] = usfs_perims_2022['Ig_Date'].dt.strftime('%Y-%m-%d')
usfs_perims_2023['Ig_Date'] = pd.to_datetime(usfs_perims_2023['DISCOVERYD'])
usfs_perims_2023['Ig_Date'] = usfs_perims_2023['Ig_Date'].dt.strftime('%Y-%m-%d')

In [ ]:
# create End_Date column that is 20 days after the ignition date
usfs_perims_2022['End_Date'] = pd.to_datetime(usfs_perims_2022['Ig_Date']) + pd.DateOffset(days=20)
usfs_perims_2022['End_Date'] = usfs_perims_2022['End_Date'].dt.strftime('%Y-%m-%d')
usfs_perims_2023['End_Date'] = pd.to_datetime(usfs_perims_2023['Ig_Date']) + pd.DateOffset(days=20)
usfs_perims_2023['End_Date'] = usfs_perims_2023['End_Date'].dt.strftime('%Y-%m-%d')

In [ ]:
usfs_perims_2023['End_Date']

In [42]:
# save to geojson
usfs_perims_2022.to_file('data/usfs_perims/usfs_perims/2022/usfs_perims_2022.geojson', driver='GeoJSON')
usfs_perims_2023.to_file('data/usfs_perims/usfs_perims/2023/usfs_perims_2023.geojson', driver='GeoJSON')

#### NIFC Perims

In [14]:
nifc_perims_2022_10k = nifc_perims_2022[nifc_perims_2022['GIS_ACRES'] > 2480]

In [ ]:
nifc_perims_2022_10k = nifc_perims_2022_10k.to_crs(4326)
nifc_perims_2022_10k['center'] = nifc_perims_2022_10k['geometry'].centroid
nifc_perims_2022_10k['lat'] = nifc_perims_2022_10k['center'].y
nifc_perims_2022_10k['lon'] = nifc_perims_2022_10k['center'].x

In [30]:
# nifc_perims_2022_10k[nifc_perims_2022_10k['DATE_CUR'] == '2022']
# drop all rows where DATE_CUR is 2022
nifc_perims_2022_10k = nifc_perims_2022_10k[nifc_perims_2022_10k['DATE_CUR'] != '2022']

In [ ]:
# Ensure DATE_CUR is a string and normalize formats
def parse_date(date):
    date = str(date)  # Ensure it's a string
    if len(date) >= 8:
        return pd.to_datetime(date[:8], format='%Y%m%d', errors='coerce').date()
    return None

# Create new column with curated date values
nifc_perims_2022_10k["date_curated"] = nifc_perims_2022_10k["DATE_CUR"].apply(parse_date)

In [ ]:
nifc_perims_2022_10k['date_curated'].sort_values(ascending=False)

In [ ]:
nifc_perims_2022_10k

#### MTBS Perims

In [17]:
mtbs_perims['YEAR'] = pd.to_datetime(mtbs_perims.Ig_Date).dt.year
mtbs_perims = mtbs_perims[mtbs_perims['Incid_Type'] != 'Prescribed Fire']

In [18]:
mtbs_perims_10k = mtbs_perims[mtbs_perims['BurnBndAc'] > 2480]

In [21]:
mtbs_perims_2022 = mtbs_perims_10k[mtbs_perims_10k['YEAR'] == 2022]
mtbs_perims_2023 = mtbs_perims_10k[mtbs_perims_10k['YEAR'] == 2023]

In [ ]:
mtbs_perims_2022['End_Date'] = pd.to_datetime(mtbs_perims_2022['Ig_Date']) + pd.DateOffset(days=20)
mtbs_perims_2022['End_Date'] = mtbs_perims_2022['End_Date'].dt.strftime('%Y-%m-%d')

mtbs_perims_2023['End_Date'] = pd.to_datetime(mtbs_perims_2023['Ig_Date']) + pd.DateOffset(days=20)
mtbs_perims_2023['End_Date'] = mtbs_perims_2023['End_Date'].dt.strftime('%Y-%m-%d')

In [45]:
# save to geojson
mtbs_perims_2022.to_file('data/mtbs_perims/mtbs_perimeter_data/mtbs_perims_2022.geojson', driver='GeoJSON')
mtbs_perims_2023.to_file('data/mtbs_perims/mtbs_perimeter_data/mtbs_perims_2023.geojson', driver='GeoJSON')

In [ ]:
mtbs_perims_2023

#### Viirs perims

In [55]:
from owslib.ogcapi.features import Features

 

OGC_URL = "https://firenrt.delta-backend.com"

 

w = Features(url=OGC_URL)

perimeters_archive_results = w.collection_items(

    "public.eis_fire_lf_perimeter_archive",  

    bbox=["-125.0", "24.396308", "-66.934570", "49.384358"],  # USA bounding box,

    datetime=["2022-01-01T00:00:00+00:00/2022-12-31T23:59:00+00:00"],

    limit=9000

)

perimeters = gpd.GeoDataFrame.from_features(perimeters_archive_results["features"],crs='EPSG:4326')

ValueError: Assigning CRS to a GeoDataFrame without a geometry column is not supported. Supply geometry using the 'geometry=' keyword argument, or by providing a DataFrame with column name 'geometry'

In [51]:
perimeters

,geometry,duration,farea,fireid,flinelen,fperim,meanfrp,n_newpixels,n_pixels,ogc_fid,pixden,t
0,"POLYGON ((-118.65750 42.41373, -118.65754 42.4...",0.0,8.789263,F8373,10.956134,11.322871,8.944839,31,31,60699,3.527031,2018-07-05T00:00:00
1,"POLYGON ((-118.65750 42.41373, -118.65754 42.4...",0.0,8.789263,F8373,0.000000,11.322871,0.000000,0,31,60700,3.527031,2018-07-18T00:00:00
2,"POLYGON ((-118.65750 42.41373, -118.65754 42.4...",0.0,8.789263,F8373,0.000000,11.322871,0.000000,0,31,60701,3.527031,2018-07-19T00:00:00
3,"POLYGON ((-118.65750 42.41373, -118.65754 42.4...",0.0,8.789263,F8373,0.000000,11.322871,0.000000,0,31,60702,3.527031,2018-07-20T00:00:00
4,"POLYGON ((-118.65750 42.41373, -118.65754 42.4...",0.0,8.789263,F8373,0.000000,11.322871,0.000000,0,31,60703,3.527031,2018-07-21T00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
4749,"MULTIPOLYGON (((-122.54543 41.87730, -122.5454...",4.5,140.445583,F8401,0.000000,60.833295,0.000000,0,906,100211,6.450897,2018-07-19T12:00:00
4750,"MULTIPOLYGON (((-122.54524 41.87721, -122.5452...",2.0,125.628340,F8401,26.704609,56.553555,32.757100,231,727,100212,5.786911,2018-07-07T12:00:00
4751,"MULTIPOLYGON (((-122.54516 41.87718, -122.5452...",1.5,111.093055,F8401,30.950403,53.319738,4.345120,166,496,100213,4.464726,2018-07-07T00:00:00
4752,"MULTIPOLYGON (((-122.54507 41.87714, -122.5451...",1.0,64.330838,F8401,19.767479,40.321269,81.707979,94,330,100214,5.129733,2018-07-06T12:00:00


In [ ]:
import datetime as dt
import pandas as pd
from owslib.ogcapi.features import Features
import geopandas as gpd
from typing import Optional

def get_usa_fire_perimeters(start_date: str, end_date: str, min_area: float = None) -> Optional[gpd.GeoDataFrame]:
    """
    Retrieve fire perimeters for the continental USA within a specified date range.
    Uses appropriate collection based on date range - snapshot for recent data (<=20 days)
    and large fire collection for historical data.
    
    Parameters:
    -----------
    start_date : str
        Start date in ISO format (YYYY-MM-DD)
    end_date : str
        End date in ISO format (YYYY-MM-DD)
    min_area : float, optional
        Minimum fire area in square kilometers to include
        
    Returns:
    --------
    gpd.GeoDataFrame
        GeoDataFrame containing fire perimeters and associated data
    """
    # Format dates for API
    start_datetime = f"{start_date}T00:00:00+00:00"
    end_datetime = f"{end_date}T23:59:59+00:00"
    
    # Initialize OGC API connection
    OGC_URL = "https://firenrt.delta-backend.com"
    w = Features(url=OGC_URL)
    
    # Set up filter conditions
    filter_conditions = []
    if min_area is not None:
        filter_conditions.append(f"farea>{min_area}")
    
    filter_str = " AND ".join(filter_conditions) if filter_conditions else None
    
    # Define USA bounding box (CONUS)
    # Format: [min_lon, min_lat, max_lon, max_lat]
    usa_bbox = ["-125.0", "24.396308", "-66.934570", "49.384358"]
    
    # Determine which collection to use based on date range
    end_date_dt = dt.datetime.strptime(end_date, "%Y-%m-%d")
    start_date_dt = dt.datetime.strptime(start_date, "%Y-%m-%d")
    date_range = (end_date_dt - start_date_dt).days
    
    # Use large fire collection for historical data
    collection_name = "public.eis_fire_lf_perimeter_nrt"
    
    # If date range is <= 20 days and includes recent dates, use snapshot collection
    twenty_days_ago = dt.datetime.now() - dt.timedelta(days=20)
    if date_range <= 20 and end_date_dt >= twenty_days_ago:
        collection_name = "public.eis_fire_snapshot_perimeter_nrt"
    
    print(f"Using collection: {collection_name}")
    
    try:
        all_features = []
        offset = 0
        limit = 1000  # Maximum number of results per request
        
        while True:
            # Query the API for fire perimeters with pagination
            perimeter_results = w.collection_items(
                collection_name,
                bbox=usa_bbox,
                datetime=[f"{start_datetime}/{end_datetime}"],
                limit=limit,
                offset=offset,
                filter=filter_str
            )
            
            # Add features to our collection
            features = perimeter_results.get("features", [])
            if not features:
                break
                
            all_features.extend(features)
            
            # Check if we've retrieved all results
            if len(all_features) >= perimeter_results["numberMatched"] or not features:
                break
                
            # Update offset for next page
            offset += len(features)
            
            # Progress update
            print(f"Retrieved {len(all_features)} of {perimeter_results['numberMatched']} fire perimeters...")
        
        if not all_features:
            print("No fire perimeters found for the specified criteria.")
            return None
            
        # Convert all features to GeoDataFrame
        perimeters = gpd.GeoDataFrame.from_features(all_features)
        
        # Set CRS
        perimeters = perimeters.set_crs("epsg:4326")
        
        # Sort by time and area
        perimeters = perimeters.sort_values(by=["t", "farea"], ascending=[False, False])
        
        # Add some useful derived columns
        perimeters['start_date'] = pd.to_datetime(perimeters['t']) - pd.to_timedelta(perimeters['duration'], unit='D')
        perimeters['end_date'] = pd.to_datetime(perimeters['t'])
        
        # Calculate additional statistics
        perimeters['perimeter_area_ratio'] = perimeters['fperim'] / perimeters['farea']
        
        print(f"\nSuccessfully retrieved {len(perimeters)} fire perimeters")
        return perimeters
        
    except Exception as e:
        print(f"Error retrieving fire perimeters: {str(e)}")
        return None

# Example usage:
if __name__ == "__main__":
    # Get fires from the last 300 days
    end_date = dt.datetime.now().strftime("%Y-%m-%d")
    start_date = (dt.datetime.now() - dt.timedelta(days=600)).strftime("%Y-%m-%d")
    # start_date = '2024-06-01'
    # end_date = '2024-07-01'
    
    # Get all fires larger than 5 sq km
    fires_df = get_usa_fire_perimeters(start_date, end_date, min_area=5)
    
    if fires_df is not None:
        print("\nLargest fires:")
        print(fires_df.nlargest(5, 'farea')[['fireid', 'farea', 'duration', 'start_date', 'end_date']])

In [ ]:
fires_df.start_date.sort_values(ascending=False)